# Part 3: DIY Linear Regression

In this part, you will complete a Do-It-Yourself (DIY) implementation of ordinary least squares (OLS) linear regression in an object-oriented pattern that corresponds with the Scikit-Learn API.

**Learning objectives.** You will:
1. Write object-oriented code for a Python class, matching standard API patterns.
2. Apply numerical Python (NumPy) to efficiently implement ordinary least squares linear regression using the closed-form solution (normal equation).
3. Evaluate your implementation compared to the Scikit-Learn standard and simple baseline models on synthetic data.
4. Explore the effects of polynomial feature expansion on model performance and overfitting.

## Background: Linear Regression Theory

Linear regression models the relationship between a dependent variable $y$ and independent variables $\mathbf{x}$ as:

$$y = \mathbf{x}^T \mathbf{w} + \epsilon$$

where $\mathbf{w}$ are the model weights and $\epsilon$ is noise.

**Normal Equation (Closed-Form Solution):**
The maximum likelihood estimate (MLE) for the weights can be found analytically:

$$\mathbf{w}^* = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$$

where $\mathbf{X}$ is the design matrix with each row being a data point.

For numerical stability, it's better to solve the linear system $\mathbf{X}^T \mathbf{X} \mathbf{w} = \mathbf{X}^T \mathbf{y}$ directly rather than computing the matrix inverse.

## Task 1

First, we will establish baseline models to which we can compare our DIY implementation. Run the following code to generate synthetic data for use in this part of the assignment. Observe that the predictive target is continuous, and that the code also splits the synthetic data into train and test sets for you.

Implement and evaluate two baseline approaches:

1. **Constant baseline**: Predict the mean of the training targets for all examples (this is the optimal constant prediction under MSE loss).
2. **Scikit-Learn baseline**: Use Scikit-Learn to fit a [linear regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) on the train set with default parameters.

For both baselines, evaluate and report the [mean squared error (MSE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) on both the train set and the test set.

In [4]:
# Run but do not modify this code

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n = 100
features = 10

# Generate synthetic data
X = np.random.normal(size=(n, features))
true_weights = np.random.normal(size=features)
noise = np.random.normal(scale=0.5, size=n)
y = X @ true_weights + noise

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2025)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Target statistics - Mean: {y_train.mean():.3f}, Std: {y_train.std():.3f}")

Training set size: (70, 10)
Test set size: (30, 10)
Target statistics - Mean: 0.241, Std: 3.236


In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Write code for task 1 here
y_train_mean = np.mean(y_train)
y_train_constant = np.full_like(y_train, y_train_mean)
y_test_constant = np.full_like(y_test, y_train_mean)
MSE_train_constant = mean_squared_error(y_train, y_train_constant)
MSE_test_constant = mean_squared_error(y_test, y_test_constant)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_train_linear_prediction = linear_model.predict(X_train)
y_test_linear_prediction = linear_model.predict(X_test)
MSE_train_linear = mean_squared_error(y_train, y_train_linear_prediction)
MSE_test_linear = mean_squared_error(y_test, y_test_linear_prediction)
print("The MSE on the train set for the constant baseline is", MSE_train_constant)
print("The MSE on the test set for the constant baseline is", MSE_test_constant)
print("The MSE on the train set for the Scikit-Learn baseline is", MSE_train_linear)
print("The MSE on the test set for the Scikit-Learn baseline is", MSE_test_linear)

The MSE on the train set for the constant baseline is 10.470370767313392
The MSE on the test set for the constant baseline is 10.471856783310528
The MSE on the train set for the Scikit-Learn baseline is 0.23159737368296537
The MSE on the test set for the Scikit-Learn baseline is 0.2722365649138697


## Task 2

Complete the following class to implement linear regression using the closed-form solution. Some important notes about the implementation:

1. The Scikit-Learn API treats an input `X` array as a matrix with a row for every data point and a column for every feature.
2. For `fit`, every row in `X` corresponds to a given output in `y`. You don't need to return anything, just compute the optimal weights (stored as instance variables).
3. For `predict`, you should return a NumPy array with one predicted value for every row in the input `X`.
4. The number of weights in your model should equal the number of features (columns) in the `X` matrix.
5. **Closed-form solution**: Use the normal equation $\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$. Use NumPy's [linalg.solve](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) instead of computing the inverse directly for better numerical stability: solve $\mathbf{X}^T \mathbf{X} \mathbf{w} = \mathbf{X}^T \mathbf{y}$.
6. For predictions, use the dot product of the input features and learned weights.
7. Include debugging information when `verbose=True` in the `fit` method (e.g., print the computed weights).
8. Use vectorized NumPy operations to avoid slow for loops over large amounts of data.
9. You do **not** need to include a bias/intercept term for this implementation.

In [7]:
class LinearRegression:
    def __init__(self, random_state=2025):
        """
        Parameters:
        -----------
        random_state : int
            Random seed for reproducibility (not used in closed-form solution)
        """
        self.random_state = random_state
        self.weights = None

    def predict(self, X):
        """Predict continuous values for each row in X"""
        # todo: complete predict method
        if self.weights is None:
            raise ValueError("Make sure to call fit(X, y, verbose) before using predict(X).")
        if X.shape[1] != self.weights.shape[0]:
            raise ValueError("Number of weights are not equal to the number of features in the X matrix.")
        return X @ self.weights
        

    def fit(self, X, y, verbose=False):
        """Fit the training data using the normal equation.
        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        y : array-like, shape = [n_examples]
          Target values (continuous).
        verbose : bool
          Whether to print training progress information.
        """
        # todo: implement closed-form solution
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y have incompatible sample sizes.")
        XTX = X.T @ X
        XTy = X.T @ y
        self.weights = np.linalg.solve(XTX, XTy)
        if verbose:
            print("X shape:", X.shape)
            print("y shape:", y.shape)
            print("XTX:", XTX)
            print("XTy:", XTy)
            print("weights:", self.weights)

## Task 3

Use your DIY `LinearRegression` class from task 2 to fit a linear regression model on the train set. Evaluate and report the [mean squared error (MSE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) on both the train set and the test set.

Compare the performance of your implementation to both baselines from Task 1 (constant model and Scikit-Learn). Your DIY implementation should achieve essentially identical performance to the Scikit-Learn implementation.

 **Note:** Due to numerical precision differences in floating-point arithmetic, small deviations in MSE values (less than 0.00001) can be considered as " identical".

In [9]:
# Write code for task 3 here
DIY = LinearRegression()
DIY.fit(X_train, y_train, verbose = False)
y_train_DIY_pred = DIY.predict(X_train)
y_test_DIY_pred = DIY.predict(X_test)
MSE_train_DIY = mean_squared_error(y_train, y_train_DIY_pred)
MSE_test_DIY = mean_squared_error(y_test, y_test_DIY_pred)
print("The MSE on the train set for the DIY model is", MSE_train_DIY)
print("The MSE on the test set for the DIY model is", MSE_test_DIY)
print("The MSE on the train set for the Scikit-Learn baseline is", MSE_train_linear)
print("The MSE on the test set for the Scikit-Learn baseline is", MSE_test_linear)
print("Are the train MSE for the DIY model and the Scikit-Learn baseline identical?", np.isclose(MSE_train_DIY, MSE_train_linear))
print("Are the test MSE for the DIY model and the Scikit-Learn baseline identical?", np.isclose(MSE_test_DIY, MSE_test_linear))

The MSE on the train set for the DIY model is 0.23159737371936245
The MSE on the test set for the DIY model is 0.27223833995486485
The MSE on the train set for the Scikit-Learn baseline is 0.23159737368296537
The MSE on the test set for the Scikit-Learn baseline is 0.2722365649138697
Are the train MSE for the DIY model and the Scikit-Learn baseline identical? True
Are the test MSE for the DIY model and the Scikit-Learn baseline identical? True


## Task 4

Now we will explore **polynomial feature expansion** to train a quadratic model and demonstrate overfitting. Create quadratic features from the original training data by computing all pairwise products of features. Specifically:

1. Generate polynomial features: For the original features $x_1, x_2, \ldots, x_d$, create all quadratic terms $x_i \cdot x_j$ for $i \leq j$. This includes squared terms like $x_1^2, x_2^2, \ldots$ and cross-product terms like $x_1 \cdot x_2, x_1 \cdot x_3, \ldots$. You can use Scikit-Learn's [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) with `degree=2` and `include_bias=False` to generate these features automatically or you can write your own NumPy code to generate these.

2. Use your DIY `LinearRegression` class to fit a model on the expanded feature set.

3. Evaluate and report the MSE on both train and test sets for the polynomial model. Compare these results to the linear model from Task 3.

4. Discuss what you observe about the training vs. test performance. Since the true underlying relationship is linear, what happens when you add quadratic features? Do you observe overfitting, underfitting, or neither?

In [11]:
from sklearn.preprocessing import PolynomialFeatures

# Write code for task 4 here
polynomial = PolynomialFeatures(degree = 2, include_bias = False)
X_train_polynomial = polynomial.fit_transform(X_train)
X_test_polynomial = polynomial.transform(X_test)
DIY_polynomial = LinearRegression()
DIY_polynomial.fit(X_train_polynomial, y_train, verbose = False)
y_train_DIY_polynomial_pred = DIY_polynomial.predict(X_train_polynomial)
y_test_DIY_polynomial_pred = DIY_polynomial.predict(X_test_polynomial)
MSE_train_DIY_polynomial = mean_squared_error(y_train, y_train_DIY_polynomial_pred)
MSE_test_DIY_polynomial = mean_squared_error(y_test, y_test_DIY_polynomial_pred)
print("The MSE on the train set for the Polynomial DIY model is", MSE_train_DIY_polynomial)
print("The MSE on the test set for the Polynomial DIY model is", MSE_test_DIY_polynomial)
print("The MSE on the train set for the DIY model is", MSE_train_DIY)
print("The MSE on the test set for the DIY model is", MSE_test_DIY)

The MSE on the train set for the Polynomial DIY model is 0.010110053760697799
The MSE on the test set for the Polynomial DIY model is 4.290069404611641
The MSE on the train set for the DIY model is 0.23159737371936245
The MSE on the test set for the DIY model is 0.27223833995486485


Looking at the train MSE for the Linear DIY model and the Polynomial DIY model, the train MSE for the Linear DIY model is 0.2316 while the train MSE for the Polynomial DIY model is 0.0101. It seems that the quadratic model drives the training error down, meaning that the quadratic model does really well with the training data compared to the simpler linear model.

But now lets look at the test MSE for the Linear DIY model and the Polynomial DIY model. The test MSE for the Linear DIY model is 0.2722 while the test MSE for the Polynomial DIY model is 4.2901. With this, it showcases that the quadratic model shoots up the test error and the test error of the quadratic model is significantly greater than the test error for the simplier linear model.

Since the Polynomial DIY model has a really low train MSE but has a higher test MSE than the linear model, this does suggest that there is overfitting behavior. Since the true underlying relationship is linear, adding extra terms in degree 2 would create many unnecessary parameters. With the quadratic model, it expanded the original features to add more features that would make the OLS saturated. Like for the training set, the quadratic model would try to bend in order to follow random noises in the data, which would lower the training error. But the problem is that the noise is not the real pattern as the OLS would fit the noise using those extra terms. So when we used the test set, the model would bend at the wrong places which would make the test error much greater and would indicate that the model is not the best for generalization.